In [ ]:
# Training on colab gpu
from google.colab import drive

drive.mount('/content/gdrive')

%cd /content/gdrive/MyDrive/Custom_Emojify/src/models/tfidf

In [ ]:
from importlib import reload
import numpy as np
import pandas as pd

import tfidf_model
import config_tfidf

%cd /content/gdrive/MyDrive/Custom_Emojify
from src.utils import pre_processing
reload(pre_processing)
%cd /content/gdrive/MyDrive/Custom_Emojify/src/models/tfidf
reload(tfidf_model)
reload(config_tfidf)
reload(pre_processing)


In [ ]:
df = pd.read_csv(config_tfidf.path_gdrive, index_col=[0])
df.Text = df.Text.apply(lambda row: pre_processing.remove_extra_spaces(row)) # remove extra spaces
df.Text = df.Text.apply(lambda row: pre_processing.hashtag_mentions_removal(row)) # Remove mentions & hashtags
(X_train, y_train), (X_val, y_val), (X_test, y_test) = pre_processing.load_and_split(df, 
                                                                                    train_ratio=0.7, 
                                                                                    validation_ratio=0.2,
                                                                                    test_ratio=0.1)

In [ ]:
dir = '/content/gdrive/MyDrive/model_weights'
model = tfidf_model.TfidfModel(dir=dir)
model.build(X_train)
model.summary()

Training with new data (16 total classes since I added in the flag and the christmas emoji again to see how large of an impact it actually has on model performance). My hunch is that this newly added data with classes will not be skewing the data too much and may actually even result in a slight increase in performance

In [ ]:
epochs=50
learning_rate=0.0085
batch_size=128

history_newmodl=model.train(X_train, y_train, X_val, y_val, epochs, learning_rate, batch_size, val_model=True)

In [ ]:
model.reset_weights()
epochs=4
learning_rate=0.00075
batch_size=128

hist_final_newmodel=model.train(X_train, y_train, X_val, y_val, epochs, learning_rate, batch_size, val_model=False) # val mode false combines the training + val data into just training

model.test(X_test, y_test, 128)

Model categorical accuracy increased by actually adding in the two classes

Everything below these cells is older training (using only 14 classes)

In [ ]:
epochs=50
learning_rate=0.0085
batch_size=128

history=model.train(X_train, y_train, X_val, y_val, epochs, learning_rate, batch_size, val_model=True)

In [ ]:
import matplotlib.pyplot as plt
# plotting function:
def generate_plots(history):
    #accuracies
    plt.plot(history.history['categorical_accuracy'])
    plt.plot(history.history['val_categorical_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()
    #loss curves
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()

In [ ]:
generate_plots(history=history)

Let's adjust some parameters and re-try this

In [ ]:
model.reset_weights()
epochs=25
learning_rate=0.00045
batch_size=256

history2=model.train(X_train, y_train, X_val, y_val, epochs, learning_rate, batch_size, val_model=True)
generate_plots(history=history2)

In [ ]:
model.reset_weights()
epochs=25
learning_rate=0.00055
batch_size=128

history3=model.train(X_train, y_train, X_val, y_val, epochs, learning_rate, batch_size, val_model=True)
generate_plots(history=history3)

### Let's combine the training + validation data into one training set and then re-train for around 4 - 5 epochs and save the model to evaluate performance with.

In [ ]:
model.reset_weights()
epochs=3
learning_rate=0.00075
batch_size=128

hist_final=model.train(X_train, y_train, X_val, y_val, epochs, learning_rate, batch_size, val_model=False) # val mode false combines the training + val data into just training

In [ ]:
model.test(x=X_test, y=y_test, batch_size=batch_size)

We've received ~34% categorical accuracy

In [ ]:
pred = model.predict(x=['Lets goooo beautiful day outside'])

In [ ]:
np.argmax(pred)

In [ ]:
model.Model.save('/content/gdrive/MyDrive/Custom_Emojify/trained_model/', save_format="tf")

In [ ]:
import tensorflow as tf
loaded_model = tf.keras.models.load_model('/content/gdrive/MyDrive/Custom_Emojify/trained_model/')

In [ ]:
np.argmax(loaded_model.predict(x=['Lets gooo beautiful day outside']))

In [ ]:
loaded_model.evaluate(X_test, y_test, 128)